# 🧼 Séance 3 : Nettoyage et Prétraitement des Données
Dataset : Amazon Products Ratings & Reviews

Dans cette séance, nous allons apprendre à :
- Identifier et traiter les valeurs manquantes
- Convertir les types de données
- Supprimer les doublons
- Nettoyer le texte
- Encoder les variables catégorielles
- Appliquer une normalisation ou standardisation

In [ ]:
# 📦 Chargement des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

In [ ]:
# Télécharger le fichier CSV directement depuis GitHub
!wget https://raw.githubusercontent.com/bidoscar/AFRICITIZEN-ACDS-Coding/main/Seance_1_Manipulation_Pandas/Datasets/amazon.csv

| Colonne             | Description en français                                                   |
|---------------------|---------------------------------------------------------------------------|
| `product_id`        | Identifiant unique du produit                                             |
| `product_name`      | Nom du produit                                                            |
| `category`          | Catégorie ou arborescence du produit sur Amazon                          |
| `discounted_price`  | Prix remisé du produit (après réduction)                                 |
| `actual_price`      | Prix initial (sans réduction)                                            |
| `discount_percentage` | Pourcentage de réduction appliqué                                     |
| `rating`            | Note moyenne donnée par les utilisateurs (sur 5)                         |
| `rating_count`      | Nombre d’utilisateurs ayant évalué le produit                            |
| `about_product`     | Description ou caractéristiques principales du produit                   |
| `user_id`           | Identifiant(s) des utilisateur(s) ayant laissé un avis                   |
| `user_name`         | Nom(s) des utilisateur(s) ayant laissé un avis                           |
| `review_id`         | Identifiant unique de l’avis utilisateur                                 |
| `review_title`      | Titre court ou résumé de l’avis                                          |
| `review_content`    | Contenu détaillé de l’avis                                                |
| `img_link`          | Lien vers l’image du produit                                              |
| `product_link`      | Lien vers la fiche produit officielle sur Amazon                         |


In [ ]:
# Chargement du fichier CSV
df = pd.read_csv('amazon.csv')
df.head()

## 🔍 1. Inspection et valeurs manquantes

In [ ]:
# Aperçu des infos générales
df.info()

In [ ]:
# Valeurs manquantes
df.isnull().sum()

## 🧹 2. Nettoyage des types de données
On va retirer ₹ et % pour convertir en float.

In [ ]:
# Créons d'abord une copie de notre jeu de données
df1 = df.copy()

In [ ]:
# On a remarqué la présence de | dans la colonne rating
# Essayons de le remplacer par une value manquant (on peut aussi décider de le remplacer avec la médiane ou la moyenne)
df1['rating'] = df1['rating'].replace('|', np.nan)

In [ ]:
# Enlevons le symbol de devise et le délimitateur de millier
df1['discounted_price'] = df1['discounted_price'].str.replace("₹", "").str.replace(",", "").astype(float)
df1['actual_price'] = df1['actual_price'].str.replace("₹", "").str.replace(",", "").astype(float)
df1['rating'] = df1['rating'].str.replace(",", "").astype(float)
df1['rating_count'] = df1['rating_count'].str.replace(",", "").astype(float)

In [ ]:
# Changeons le type de donnée et ramenons le pourcentage en proportion (entre 0 et 1) dans discount_Percentage
df1['discount_percentage'] = df1['discount_percentage'].str.replace('%','').astype('float64')
df1['discount_percentage'] = df1['discount_percentage'] / 100

## 📑 3.1 Gestion des valeurs manquantes

In [ ]:
# Une copy de nos donnees
df2 = df1.copy()

In [ ]:
# Valeurs manquantes
df2.isnull().sum()

In [ ]:
# Verifions si la variable est normalement distribuee
sns.histplot(df2['rating_count'], kde=True, color='green')
plt.title('Histogramme avec densité (seaborn)')
plt.xlabel("Nombre d'Evaluation")
plt.show()

In [ ]:
df2['rating_count'].describe()

In [ ]:
# Suppression ou imputation des valeurs manquantes
# df1['rating_count'].fillna(df1['rating_count'].median(), inplace=True)
df2.fillna({'rating_count': df2['rating_count'].median()}, inplace=True)

In [ ]:
# Verifions si la variable est normalement distribuee
sns.histplot(df2['rating'], kde=True, color='green')
plt.title('Histogramme avec densité (seaborn)')
plt.xlabel('Evaluation')
plt.show()

In [ ]:
# Comparons le moyenne a la mediane
df2['rating'].describe()

In [ ]:
# Imputation des valeurs manquantes de rating par sa moyenne
# df2['rating'].fillna(df2['rating'].mean(), inplace=True)
# df2.fillna({'rating': df2['rating'].mean()}, inplace=True)

In [ ]:
# verifions la ligne ayant la valeur manquante
df2[df2['rating'].isnull()]

In [ ]:
index_du_nan = df2[df2['rating'].isnull()].index.tolist()
index_du_nan

In [ ]:
df2.loc[index_du_nan[0], 'product_name']

In [ ]:
df2.fillna({'rating': 3.9}, inplace=True)

In [ ]:
# Valeurs manquantes
df2.isnull().sum()

## 📑 3.2 Gestion des doublons

In [ ]:
# Find Duplicate
df2.duplicated().any()

In [ ]:
df2.columns

In [ ]:
#df2.duplicated(subset=['product_id', 'product_name', 'rating'])
df2.duplicated(subset=['product_id', 'product_name', 'rating', 'rating_count', 'about_product']).any()

In [ ]:
df2[df2.duplicated(subset=['product_id', 'product_name', 'rating', 'rating_count', 'about_product'])]

In [ ]:
df2[df2.duplicated(subset=['product_id', 'product_name', 'rating', 'rating_count', 'about_product'])]['product_id'].value_counts()

In [ ]:
# Supprimer les doublons partiels (en gardant la première occurrence uniquement)
df3 = df2.drop_duplicates(subset=['product_id', 'product_name', 'rating', 'rating_count', 'about_product'], keep='first')

In [ ]:
# Supprimer les colonnes indesirables
df3.drop(['product_name', 'about_product', 'user_id', 'user_name', 'review_id', 'review_title', 'review_content', 'img_link', 'product_link'], axis=1, inplace=True)

In [ ]:
df3.head()

## ✏️ 4.1 Nettoyage des chaînes de caractères

In [ ]:
# Mise en minuscules et suppression espaces
df3['product_id'] = df3['product_id'].str.lower().str.strip()
df3['category'] = df3['category'].str.lower().str.strip()

## ✏️ 4.2 Nettoyage des chaînes de caractères

In [ ]:
df3['category'].value_counts()

In [ ]:
df3['category'][0]

In [ ]:
df3.sample(n=1)['category'].values

In [ ]:
# Séparer la chaîne 'category' selon le séparateur "|"
category_split = df3['category'].str.split('|', expand=True)
category_split

In [ ]:
# Renommer les colonnes résultantes
category_split.columns = [f'category_level_{i+1}' for i in range(category_split.shape[1])]
category_split.head()

In [ ]:
category_split.isnull().sum()/len(category_split)*100

In [ ]:
# remplissage en avant (forward fill) ligne par ligne.
category_split = category_split.ffill(axis=1)
category_split.head()

In [ ]:
category_split['category_level_1'].value_counts()

In [ ]:
category_split['category_level_2'].value_counts()

In [ ]:
category_split['category_level_3'].value_counts()

In [ ]:
category_split['category_level_7'].value_counts()

In [ ]:
# Créer une série des fréquences
value_counts = category_split['category_level_1'].value_counts()

# Identifier les catégories rares (< 10 occurrences)
categories_rares = value_counts[value_counts < 10].index

# Remplacer ces modalités par 'autres'
category_split['category_level_1'] = category_split['category_level_1'].replace(categories_rares, 'others')

In [ ]:
category_split['category_level_1'].value_counts()

In [ ]:
# Remplacer ces modalités par 'autres'
category_split['category_level_1'] = category_split['category_level_1'].replace('others', 'officeproducts&other').replace('officeproducts', 'officeproducts&other')

In [ ]:
category_split['category_level_1'].value_counts()

In [ ]:
# Ajouter les nouvelles colonnes au DataFrame d'origine
df4 = pd.concat([df3, category_split['category_level_1']], axis=1)
df4.head()

In [ ]:
# Profitons pour recreer la variable categorielle de rating qu'on avait creee pendant le seance derniere
# Create a new categorical variable from rating
def classify_rating(r):
    if r <= 3.9:
        return "Medium"
    else:
        return "High"

df4['rating_level'] = df4['rating'].apply(classify_rating)
df4['rating_level'].value_counts()

## 🔠 5. Encodage des variables catégorielles

In [ ]:
df4['category_level_1'].value_counts()

In [ ]:
le = LabelEncoder()
df4['category_level_1_encoded'] = le.fit_transform(df4['category_level_1'])
df4['category_level_1_encoded'].value_counts()

In [ ]:
pd.crosstab(df4['category_level_1_encoded'], df4['category_level_1'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
encoded = ohe.fit_transform(df4[['category_level_1']])

In [ ]:
# Get column names from the encoder
col_names = ohe.get_feature_names_out(['category_level_1'])

# Convert the sparse matrix to a DataFrame
encoded_df = pd.DataFrame(encoded.toarray(), columns=col_names)

# Display the first rows
encoded_df.head()

In [ ]:
# Ajouter les nouvelles colonnes au DataFrame d'origine
df5 = pd.concat([df4, encoded_df], axis=1)
df5.head()

In [ ]:
encoded = ohe.fit_transform(df4[['rating_level']])
# Get column names from the encoder
col_names = ohe.get_feature_names_out(['rating_level'])

# Convert the sparse matrix to a DataFrame
encoded_df = pd.DataFrame(encoded.toarray(), columns=col_names)

# Ajouter les nouvelles colonnes au DataFrame d'origine
df6 = pd.concat([df5, encoded_df], axis=1)
df6.head()

## 📏 6. Normalisation et standardisation

La mise à l’échelle des données est essentielle pour de nombreux algorithmes de machine learning, en particulier ceux qui sont sensibles à la magnitude des valeurs numériques.

🎯 **1. Éviter les biais dus aux unités de mesure**
* Si une variable est exprimée en euros (ex. prix = 1000) et une autre en note (ex. rating = 4.5), un algorithme non normalisé pourrait accorder trop d’importance à la variable qui a les plus grandes valeurs numériques (même si elle est moins importante).

⚖️ **2. Équilibrer l’influence des variables**
* KNN, SVM, régression logistique, réseaux de neurones, etc. utilisent des distances ou des produits scalaires.

* Si les variables sont sur des échelles différentes, cela fausse la distance ou l’optimisation, car une variable "domine" les autres.

⚙️ **3. Améliorer la convergence des modèles**
* Les algorithmes comme la descente de gradient (utilisée dans les régressions ou réseaux de neurones) convergent plus vite et plus efficacement si les données sont centrées et réduites.

* Sinon, l’algorithme oscille et apprend plus lentement.

In [ ]:
# une autre copy des donnees
df7 = df6.copy()

In [ ]:
# Appliquer Normalisation (Min-Max Scaling)
minmax_scaler = MinMaxScaler()
df7['discounted_price_minmax'] = minmax_scaler.fit_transform(df7[['discounted_price']])

# Appliquer Standardisation (Z-score Scaling)
standard_scaler = StandardScaler()
df7['discounted_price_zscore'] = standard_scaler.fit_transform(df7[['discounted_price']])

In [ ]:
df7[['discounted_price', 'discounted_price_minmax', 'discounted_price_zscore']].describe()

In [ ]:
# Standardisation des prix
scaler = StandardScaler()
df6[['discounted_price_scaled', 'actual_price_scaled', 'rating_count_scaled', 'discount_percentage_scaled', 'rating_scaled']] = scaler.fit_transform(df6[['discounted_price', 'actual_price', 'rating_count', 'discount_percentage', 'rating']])

## 7 Gerer les valeurs aberantes

In [ ]:
sns.boxplot(x=df6['discounted_price_scaled'], color='lightcoral')
plt.title('Boxplot des prix remises (seaborn)')
plt.show()

In [ ]:
mean_dis_price = df6['discounted_price_scaled'].mean()
std_dis_price = df6['discounted_price_scaled'].std()

lower_bound = mean_dis_price - 1 * std_dis_price
upper_bound = mean_dis_price + 1 * std_dis_price

# Supprimer les valeurs en dehors de ces bornes
df_clean = df6[(df6['discounted_price_scaled'] >= lower_bound) & (df6['discounted_price_scaled'] <= upper_bound)]

In [ ]:
sns.boxplot(x=df_clean['discounted_price_scaled'], color='lightcoral')
plt.title('Boxplot des prix remises (seaborn)')
plt.show()

Suppression basée sur l’IQR (méthode boxplot)

In [ ]:
Q1 = df6['discounted_price_scaled'].quantile(0.25)
Q3 = df6['discounted_price_scaled'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 0.5 * IQR
upper_bound = Q3 + 0.5 * IQR

# Supprimer les valeurs en dehors de ces bornes
df_clean = df6[(df6['discounted_price_scaled'] >= lower_bound) & (df6['discounted_price_scaled'] <= upper_bound)]

In [ ]:
sns.boxplot(x=df_clean['discounted_price_scaled'], color='lightcoral')
plt.title('Boxplot des prix remises (seaborn)')
plt.show()

In [ ]:
df_clean.columns

In [ ]:
from scipy.stats.mstats import winsorize

df7['discounted_price_scaled_winsor'] = winsorize(df6['discounted_price_scaled'], limits=[0.01, 0.2])


Winsorization (remplacement par des bornes extrêmes)

In [ ]:
sns.boxplot(x=df7['discounted_price_scaled_winsor'], color='lightcoral')
plt.title('Boxplot des prix remises (seaborn)')
plt.show()

In [ ]:
df7['discounted_price_log'] = np.log1p(df6['discounted_price'])  # log(1 + x)

In [ ]:
sns.boxplot(x=df7['discounted_price_log'], color='lightcoral')
plt.title('Boxplot des prix remises (seaborn)')
plt.show()

## ✅ Données prêtes à l’usage !